# Import requirements

In [1]:
import os
import random
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(f'Device: {DEVICE}')

Device: cuda


# Data Augmentation Using `transforms`

In [2]:
transform_train = transforms.Compose([transforms.Resize((32, 32)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(10),
                                      transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
                                      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                     ])

transform_test = transforms.Compose([transforms.Resize((32, 32)),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                 ])

train_dataset = datasets.CIFAR10(root='../data/CIFAR_10',
                                 train=True,
                                 download=True,)
test_dataset = datasets.CIFAR10(root='../data/CIFAR_10',
                                train=False,
                                transform=transform_test)


Extracting ../data/CIFAR_10/cifar-10-python.tar.gz to ../data/CIFAR_10


In [3]:
for (image, label) in train_dataset:
    image = transforms.ToTensor()(image).unsqueeze_(0)
    print(image.shape)
    print(label)
    break

torch.Size([1, 3, 32, 32])
6


In [4]:
train_list = []
for (image, label) in tqdm(train_dataset):
    img0 = image
    img1 = transforms.RandomHorizontalFlip(1)(image)
    img2 = transforms.RandomRotation(10)(image)
    img3 = transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2))(image)
    img4 = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2)(image)
    imgs = [img0, img1, img2, img3, img4]
    
    for img in imgs:
        img = transforms.ToTensor()(img).unsqueeze_(0)
        img = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(img)
        train_list.append((img, label))

print(f'\n{len(train_list)}')

100%|██████████| 50000/50000 [01:30<00:00, 550.55it/s]


250000


In [5]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_list):
        self.data = data_list
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx][0].view(3, 32, 32)
        label = self.data[idx][1]
        return image, label

train_dataset = MyDataset(train_list)

In [6]:
BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

# Define the Convolution Neural Network (CNN)

In [7]:
class ConvNet(nn.Module):
    # similar with VGG-16
    def __init__(self):
        super(ConvNet, self).__init__()
        # input shape = (32, 32)
        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1, bias=False), # (32, 32)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1, bias=False), # (32, 32)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # (16, 16)
            
            nn.Conv2d(64, 128, 3, padding=1, bias=False), # (16, 16)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1, bias=False), # (16, 16)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # (8, 8)
            
            nn.Conv2d(128, 256, 3, padding=1, bias=False), # (8, 8)
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1, bias=False), # (8, 8)
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1, bias=False), # (8, 8)
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # (4, 4)

            nn.Conv2d(256, 512, 3, padding=1, bias=False), # (4, 4)
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1, bias=False), # (4, 4)
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1, bias=False), # (4, 4)
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # (2, 2)

            # nn.Conv2d(512, 512, 3, padding=1, bias=False), # (2, 2)
            # nn.BatchNorm2d(512),
            # nn.ReLU(),
            # nn.Conv2d(512, 512, 3, padding=1, bias=False), # (4, 4)
            # nn.BatchNorm2d(512),
            # nn.ReLU(),
            # nn.Conv2d(512, 512, 3, padding=1, bias=False), # (4, 4)
            # nn.BatchNorm2d(512),
            # nn.ReLU(),
            # nn.MaxPool2d(2, 2), # (1, 1)
            
        )


        self.clssify = nn.Sequential(
            nn.Linear(2*2*512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 10),
            nn.LogSoftmax(dim=1),
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 2*2*512)
        x = self.clssify(x)
        return x

# Define the train, evaluation

In [8]:
def train(model, train_loader, optimizer, log_interval):
    model.train()
    train_loss = 0
    correct = 0

    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % log_interval == 0 or (batch_idx + 1) == len(train_loader):
            pct = 100 * batch_idx / len(train_loader) # percent
            train_loss /= log_interval
            print(f'Train Epoch: {Epoch} [{batch_idx * len(image)}/{len(train_loader.dataset)} ({pct:.0f}%)]\tAverage Train Loss: {train_loss:.6f}')
            train_loss = 0


def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(label.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100 * correct / len(test_loader.dataset)

    return test_loss, test_accuracy

# set seeds

In [9]:
def fix_seeds(seed = 42, use_torch=False):
    # fix the seed for reproducibility 
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

    if use_torch: 
        torch.manual_seed(seed) 
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

In [10]:
def init_weights(m):
    # initialize the weight, bias
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_uniform_(m.weight.data)
        if m.bias is not None:
            torch.nn.init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight.data, mean=1, std=0.02)
        torch.nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight.data)
        torch.nn.init.normal_(m.bias.data)

In [12]:
SEED = 42
EPOCHS = 20

fix_seeds(seed=SEED, use_torch=True)
model = ConvNet().to(device=DEVICE)
model.apply(init_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


for Epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval=1500)
    test_loss, test_acc = evaluate(model, test_loader)
    print(f'\nEpoch: {Epoch}')
    print(f'Average Test Loss: {test_loss:.4f}')
    print(f'Test Accuracy: {test_acc:.2f}\n')
    torch.save(model, f'./models/model_{Epoch:02d}.pt')

Train Epoch: 1 [47968/250000 (19%)]	Average Train Loss: 1.795974
Train Epoch: 1 [95968/250000 (38%)]	Average Train Loss: 1.248161
Train Epoch: 1 [143968/250000 (58%)]	Average Train Loss: 0.995478
Train Epoch: 1 [191968/250000 (77%)]	Average Train Loss: 0.835888
Train Epoch: 1 [239968/250000 (96%)]	Average Train Loss: 0.723803
Train Epoch: 1 [124992/250000 (100%)]	Average Train Loss: 0.138366

Epoch: 1
Average Test Loss: 0.0196
Test Accuracy: 79.44

Train Epoch: 2 [47968/250000 (19%)]	Average Train Loss: 0.606333
Train Epoch: 2 [95968/250000 (38%)]	Average Train Loss: 0.553862
Train Epoch: 2 [143968/250000 (58%)]	Average Train Loss: 0.511143
Train Epoch: 2 [191968/250000 (77%)]	Average Train Loss: 0.459940
Train Epoch: 2 [239968/250000 (96%)]	Average Train Loss: 0.428601
Train Epoch: 2 [124992/250000 (100%)]	Average Train Loss: 0.085365

Epoch: 2
Average Test Loss: 0.0138
Test Accuracy: 85.53

Train Epoch: 3 [47968/250000 (19%)]	Average Train Loss: 0.356985
Train Epoch: 3 [95968/250000 

In [ ]:
model = torch.load(f'./models/model_20.pt')
for Epoch in range(21, 51):
    train(model, train_loader, optimizer, log_interval=1500)
    test_loss, test_acc = evaluate(model, test_loader)
    print(f'\nEpoch: {Epoch}')
    print(f'Average Test Loss: {test_loss:.4f}')
    print(f'Test Accuracy: {test_acc:.2f}\n')
    torch.save(model, f'./models/model_{Epoch:02d}.pt')

Train Epoch: 21 [47968/250000 (19%)]	Average Train Loss: 0.022305
Train Epoch: 21 [95968/250000 (38%)]	Average Train Loss: 0.019942
Train Epoch: 21 [143968/250000 (58%)]	Average Train Loss: 0.020935
Train Epoch: 21 [191968/250000 (77%)]	Average Train Loss: 0.020250
Train Epoch: 21 [239968/250000 (96%)]	Average Train Loss: 0.020705
Train Epoch: 21 [124992/250000 (100%)]	Average Train Loss: 0.004223

Epoch: 21
Average Test Loss: 0.0214
Test Accuracy: 89.91

Train Epoch: 22 [47968/250000 (19%)]	Average Train Loss: 0.022674
Train Epoch: 22 [95968/250000 (38%)]	Average Train Loss: 0.020973
Train Epoch: 22 [143968/250000 (58%)]	Average Train Loss: 0.019665
Train Epoch: 22 [191968/250000 (77%)]	Average Train Loss: 0.019950
Train Epoch: 22 [239968/250000 (96%)]	Average Train Loss: 0.018924
Train Epoch: 22 [124992/250000 (100%)]	Average Train Loss: 0.004508

Epoch: 22
Average Test Loss: 0.0217
Test Accuracy: 89.89

Train Epoch: 23 [47968/250000 (19%)]	Average Train Loss: 0.022718
Train Epoch: 2